In [19]:
train.data <- read.csv(file.path("..", "data", "training_data.csv"))
test.data <- read.csv(file.path("..", "data", "test_data.csv"))
sampleSubmission <- read.csv(file.path("..", "data", "sampleSubmission.csv"))

In [20]:
#Some information about the training data : 

# Sample size = 716 molecules smelled and a response is given based on 11788 predictors 
# Total dimension = X : 716x11788 = 8440208
#                   Y : 1 x 11788 = 11788

#FEATURES 
# X1 : name = intensity   ;   meaning = intensity of the odor   ; type = Factor with 2 levels : "high" (1) or "low" (2)
# X2 : name = SWEETORSOUR ;   meaning = more sweet than sour ?  ; type = logi "FALSE" or "TRUE (more sweet than sour)"
# X3-X11788 : name = V    ; meaning = physicochemical features  ; type = int (0 or 1) : have the feature or not ?


#RESPONSE 
#Each of the 716 molecules were smelled and rated on their pleasantness
# Y : name = VALENCE.PLEASANTNESS ;  meaning = how pleasant the odor is  ; type = int (0-100) : score for the smell
